# 📍 Récupération des coordonnées GPS de villes françaises via l’API Nominatim
Ce notebook utilise l'API [Nominatim](https://nominatim.org/) pour récupérer les coordonnées GPS de plusieurs villes françaises.
Il exporte ensuite les résultats dans un fichier CSV et affiche les villes sur une carte avec Plotly.

In [ ]:
import requests
import pandas as pd
import plotly.express as px

In [ ]:
# Liste des villes à traiter
city = [
    "Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris",
    "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar",
    "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
    "Moustiers-Sainte-Marie", "Bormes les Mimosas", "Cassis", "Marseille",
    "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes",
    "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege",
    "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"
]

In [ ]:
# URL de base de l'API Nominatim
url = "https://nominatim.openstreetmap.org/search?country=France&format=json&city="

# Headers requis par l'API (inclure un User-Agent personnalisé)
headers = {
    "User-Agent": "MyApp/1.0 (ristou@free.fr)"
}

In [ ]:
# Récupération des données de chaque ville
rows = []

for i in city:
    print(f"📍 Traitement de : {i}")
    r = requests.get(url + i, headers=headers)
    
    if r.status_code != 200:
        print(f"❌ Erreur HTTP {r.status_code} pour {i}")
        continue

    data = r.json()
    if not data:
        print(f"⚠️ Aucune donnée trouvée pour {i}")
        continue

    result = data[0]
    result["ville"] = i
    rows.append(result)

In [ ]:
# Création du DataFrame
df = pd.DataFrame(rows)
df = df[["ville", "display_name", "lat", "lon", "importance"]]  # importance = notoriété du lieu

# Conversion des coordonnées en float
df["lat"] = df["lat"].astype(float)
df["lon"] = df["lon"].astype(float)

# Export CSV
df.to_csv("coordonnees_villes.csv", index=False, encoding="utf-8")
print("✅ Fichier 'coordonnees_villes.csv' exporté avec succès")

## 🌍 Visualisation interactive avec Plotly

In [ ]:
fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    text="ville",
    hover_name="display_name",
    zoom=5,
    height=600,
    title="Carte des villes en France"
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_traces(marker=dict(size=12, color="red"))

fig.show()